In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
print(spark)
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
df1 = spark.read.csv("pune_home_prices.csv",header='true',inferSchema='true')
df1.show(5)

+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|           area_type| availability|            location|     size|society|total_sqft|bath|balcony|price|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|Super built-up  Area|       19-Dec|Electronic City P...|    2 BHK|Coomee |      1056|   2|      1|39.07|
|          Plot  Area|Ready To Move|    Chikka Tirupathi|4 Bedroom|Theanmp|      2600|   5|      3|120.0|
|      Built-up  Area|Ready To Move|         Uttarahalli|    3 BHK|   null|      1440|   2|      3| 62.0|
|Super built-up  Area|Ready To Move|  Lingadheeranahalli|    3 BHK|Soiewre|      1521|   3|      1| 95.0|
|Super built-up  Area|Ready To Move|            Kothanur|    2 BHK|   null|      1200|   2|      1| 51.0|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
only showing top 5 rows



In [3]:
print("(",df1.count(),",", len(df1.columns),")")
df1.printSchema()

( 13320 , 9 )
root
 |-- area_type: string (nullable = true)
 |-- availability: string (nullable = true)
 |-- location: string (nullable = true)
 |-- size: string (nullable = true)
 |-- society: string (nullable = true)
 |-- total_sqft: string (nullable = true)
 |-- bath: integer (nullable = true)
 |-- balcony: integer (nullable = true)
 |-- price: double (nullable = true)



In [4]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [5]:
df1.shape()

(13320, 9)

In [6]:
df1.select('area_type','availability','location').show(5)

+--------------------+-------------+--------------------+
|           area_type| availability|            location|
+--------------------+-------------+--------------------+
|Super built-up  Area|       19-Dec|Electronic City P...|
|          Plot  Area|Ready To Move|    Chikka Tirupathi|
|      Built-up  Area|Ready To Move|         Uttarahalli|
|Super built-up  Area|Ready To Move|  Lingadheeranahalli|
|Super built-up  Area|Ready To Move|            Kothanur|
+--------------------+-------------+--------------------+
only showing top 5 rows



In [7]:
df1.select('area_type').distinct().show()

+--------------------+
|           area_type|
+--------------------+
|      Built-up  Area|
|Super built-up  Area|
|          Plot  Area|
|        Carpet  Area|
+--------------------+



In [8]:
df1.groupBy('area_type').count().show()

+--------------------+-----+
|           area_type|count|
+--------------------+-----+
|      Built-up  Area| 2418|
|Super built-up  Area| 8790|
|          Plot  Area| 2025|
|        Carpet  Area|   87|
+--------------------+-----+



In [9]:
df1.groupBy('balcony').count().show()

+-------+-----+
|balcony|count|
+-------+-----+
|   null|  609|
|      1| 4897|
|      3| 1672|
|      2| 5113|
|      0| 1029|
+-------+-----+



In [10]:
df2 = df1.select('location','size','total_sqft','bath','price')
df2.shape()

(13320, 5)

In [11]:
df2.show(5)

+--------------------+---------+----------+----+-----+
|            location|     size|total_sqft|bath|price|
+--------------------+---------+----------+----+-----+
|Electronic City P...|    2 BHK|      1056|   2|39.07|
|    Chikka Tirupathi|4 Bedroom|      2600|   5|120.0|
|         Uttarahalli|    3 BHK|      1440|   2| 62.0|
|  Lingadheeranahalli|    3 BHK|      1521|   3| 95.0|
|            Kothanur|    2 BHK|      1200|   2| 51.0|
+--------------------+---------+----------+----+-----+
only showing top 5 rows



In [12]:
df2.filter("location is null").show()

+--------+-----+----------+----+-----+
|location| size|total_sqft|bath|price|
+--------+-----+----------+----+-----+
|    null|3 BHK|      1600|   3| 86.0|
+--------+-----+----------+----+-----+



In [13]:
df2.filter("size is null").show(5)

+--------------+----+-----------+----+------+
|      location|size| total_sqft|bath| price|
+--------------+----+-----------+----+------+
|Sarjapur  Road|null|1200 - 2400|null|34.185|
|      IVC Road|null|2000 - 5634|null| 124.0|
|  Banashankari|null|       2400|null| 460.0|
|Sarjapur  Road|null|1200 - 2400|null|28.785|
|   Devanahalli|null|1500 - 2400|null|  46.8|
+--------------+----+-----------+----+------+
only showing top 5 rows



In [14]:
df3 = df2.dropna(thresh=3,subset=('location','size','bath'))  

df3.show(5)

+--------------------+---------+----------+----+-----+
|            location|     size|total_sqft|bath|price|
+--------------------+---------+----------+----+-----+
|Electronic City P...|    2 BHK|      1056|   2|39.07|
|    Chikka Tirupathi|4 Bedroom|      2600|   5|120.0|
|         Uttarahalli|    3 BHK|      1440|   2| 62.0|
|  Lingadheeranahalli|    3 BHK|      1521|   3| 95.0|
|            Kothanur|    2 BHK|      1200|   2| 51.0|
+--------------------+---------+----------+----+-----+
only showing top 5 rows



In [15]:
df3.shape()

(13246, 5)

In [16]:
df3.select('size').distinct().show()

+----------+
|      size|
+----------+
|    14 BHK|
| 5 Bedroom|
|     7 BHK|
|    16 BHK|
| 1 Bedroom|
| 9 Bedroom|
| 3 Bedroom|
|    27 BHK|
|      1 RK|
|10 Bedroom|
|    11 BHK|
|     4 BHK|
|18 Bedroom|
| 6 Bedroom|
|     2 BHK|
| 4 Bedroom|
| 2 Bedroom|
|     6 BHK|
|     8 BHK|
|    13 BHK|
+----------+
only showing top 20 rows



In [17]:
#df3.select('size').split(' ')[0]
import pyspark.sql.functions as f
df3.select("location","size","total_sqft","bath","price",f.split("size"," ")[0].alias("bhk")).show(5)


+--------------------+---------+----------+----+-----+---+
|            location|     size|total_sqft|bath|price|bhk|
+--------------------+---------+----------+----+-----+---+
|Electronic City P...|    2 BHK|      1056|   2|39.07|  2|
|    Chikka Tirupathi|4 Bedroom|      2600|   5|120.0|  4|
|         Uttarahalli|    3 BHK|      1440|   2| 62.0|  3|
|  Lingadheeranahalli|    3 BHK|      1521|   3| 95.0|  3|
|            Kothanur|    2 BHK|      1200|   2| 51.0|  2|
+--------------------+---------+----------+----+-----+---+
only showing top 5 rows



In [18]:
df3.shape()

(13246, 5)

In [19]:
df4=df3.select("location","size","total_sqft","bath","price",f.split("size"," ")[0].alias("bhk"))

In [20]:
df4.shape()

(13246, 6)

In [21]:
df4.select("bhk").distinct().show()

+---+
|bhk|
+---+
|  7|
| 11|
|  3|
|  8|
| 16|
| 43|
|  5|
| 18|
| 27|
|  6|
| 19|
|  9|
|  1|
| 10|
|  4|
| 12|
| 13|
| 14|
|  2|
+---+



In [22]:
df4.filter(df4.bhk>20).show()

+--------------------+----------+----------+----+-----+---+
|            location|      size|total_sqft|bath|price|bhk|
+--------------------+----------+----------+----+-----+---+
|2Electronic City ...|    27 BHK|      8000|  27|230.0| 27|
|         Munnekollal|43 Bedroom|      2400|  40|660.0| 43|
+--------------------+----------+----------+----+-----+---+



In [23]:
df4.select("total_sqft").distinct().show(5)

+----------+
|total_sqft|
+----------+
|      1436|
|      1090|
|       675|
|      1512|
|      3606|
+----------+
only showing top 5 rows



In [24]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [25]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [26]:
from pyspark.sql.functions import col

In [27]:
is_float = spark.udf.register("is_float", is_float)
convert_sqft_to_num = spark.udf.register("convert_sqft_to_num", convert_sqft_to_num)

In [28]:
df4.select(is_float(col("total_sqft"))).show(4)

+--------------------+
|is_float(total_sqft)|
+--------------------+
|                true|
|                true|
|                true|
|                true|
+--------------------+
only showing top 4 rows



In [29]:
df4.filter(df4.location == 'Yelahanka').show(3)

+---------+---------+-----------+----+-----+---+
| location|     size| total_sqft|bath|price|bhk|
+---------+---------+-----------+----+-----+---+
|Yelahanka|    4 BHK|2100 - 2850|   4|186.0|  4|
|Yelahanka|1 Bedroom|        660|   1| 48.0|  1|
|Yelahanka|    3 BHK|       1600|   2| 75.0|  3|
+---------+---------+-----------+----+-----+---+
only showing top 3 rows



In [30]:
df4.filter(("location == 'Yelahanka' and bhk in ('4','2')")).show(5)

+---------+-----+-----------+----+-----+---+
| location| size| total_sqft|bath|price|bhk|
+---------+-----+-----------+----+-----+---+
|Yelahanka|4 BHK|2100 - 2850|   4|186.0|  4|
|Yelahanka|2 BHK|       1326|   2| 78.0|  2|
|Yelahanka|2 BHK|       1327|   2| 98.0|  2|
|Yelahanka|2 BHK|       1350|   2|55.55|  2|
|Yelahanka|2 BHK|       1035|   2| 45.0|  2|
+---------+-----+-----------+----+-----+---+
only showing top 5 rows



In [31]:
df4.filter(f.length("total_sqft")>7).show(4)

+------------------+-----+-----------+----+------+---+
|          location| size| total_sqft|bath| price|bhk|
+------------------+-----+-----------+----+------+---+
|         Yelahanka|4 BHK|2100 - 2850|   4| 186.0|  4|
|            Hebbal|4 BHK|3067 - 8156|   4| 477.0|  4|
|8th Phase JP Nagar|2 BHK|1042 - 1105|   2|54.005|  2|
|          Sarjapur|2 BHK|1145 - 1340|   2| 43.49|  2|
+------------------+-----+-----------+----+------+---+
only showing top 4 rows



In [32]:
df5=df4.select("location","size",convert_sqft_to_num(col("total_sqft")).alias("total_sqft"),"bath","price","bhk")

In [33]:
df5.show(3)

+--------------------+---------+----------+----+-----+---+
|            location|     size|total_sqft|bath|price|bhk|
+--------------------+---------+----------+----+-----+---+
|Electronic City P...|    2 BHK|    1056.0|   2|39.07|  2|
|    Chikka Tirupathi|4 Bedroom|    2600.0|   5|120.0|  4|
|         Uttarahalli|    3 BHK|    1440.0|   2| 62.0|  3|
+--------------------+---------+----------+----+-----+---+
only showing top 3 rows



In [34]:
df5.shape()

(13246, 6)

In [35]:
df5.filter(f.isnull("total_sqft")).count()

46

In [36]:
df5.filter(f.isnull("total_sqft") == False).count()

13200

In [37]:
df6=df5.filter(f.isnull("total_sqft") == False)


In [38]:
df6.shape()

(13200, 6)

In [39]:
df6.show(4)

+--------------------+---------+----------+----+-----+---+
|            location|     size|total_sqft|bath|price|bhk|
+--------------------+---------+----------+----+-----+---+
|Electronic City P...|    2 BHK|    1056.0|   2|39.07|  2|
|    Chikka Tirupathi|4 Bedroom|    2600.0|   5|120.0|  4|
|         Uttarahalli|    3 BHK|    1440.0|   2| 62.0|  3|
|  Lingadheeranahalli|    3 BHK|    1521.0|   3| 95.0|  3|
+--------------------+---------+----------+----+-----+---+
only showing top 4 rows



In [40]:
df7 = df6.withColumn("price_per_sqft", df6.price * 100000 / df6.total_sqft)    #df6['price']*100000/df6['total_sqft']

In [41]:
df7.show(5)

+--------------------+---------+----------+----+-----+---+-----------------+
|            location|     size|total_sqft|bath|price|bhk|   price_per_sqft|
+--------------------+---------+----------+----+-----+---+-----------------+
|Electronic City P...|    2 BHK|    1056.0|   2|39.07|  2|3699.810606060606|
|    Chikka Tirupathi|4 Bedroom|    2600.0|   5|120.0|  4|4615.384615384615|
|         Uttarahalli|    3 BHK|    1440.0|   2| 62.0|  3|4305.555555555556|
|  Lingadheeranahalli|    3 BHK|    1521.0|   3| 95.0|  3|6245.890861275477|
|            Kothanur|    2 BHK|    1200.0|   2| 51.0|  2|           4250.0|
+--------------------+---------+----------+----+-----+---+-----------------+
only showing top 5 rows



In [42]:
stats=df7.describe("price_per_sqft")

In [43]:
stats.show()

+-------+------------------+
|summary|    price_per_sqft|
+-------+------------------+
|  count|             13200|
|   mean| 7920.759149939642|
| stddev|106727.15520931015|
|    min|267.82981328435875|
|    max|             1.2E7|
+-------+------------------+



In [44]:
loc_remove_space=df7.withColumn("locations",f.regexp_replace("location",'^\s+$',''))   #df7.location.apply(lambda x: x.strip())

In [45]:
loc_remove_space.show(4)

+--------------------+---------+----------+----+-----+---+-----------------+--------------------+
|            location|     size|total_sqft|bath|price|bhk|   price_per_sqft|           locations|
+--------------------+---------+----------+----+-----+---+-----------------+--------------------+
|Electronic City P...|    2 BHK|    1056.0|   2|39.07|  2|3699.810606060606|Electronic City P...|
|    Chikka Tirupathi|4 Bedroom|    2600.0|   5|120.0|  4|4615.384615384615|    Chikka Tirupathi|
|         Uttarahalli|    3 BHK|    1440.0|   2| 62.0|  3|4305.555555555556|         Uttarahalli|
|  Lingadheeranahalli|    3 BHK|    1521.0|   3| 95.0|  3|6245.890861275477|  Lingadheeranahalli|
+--------------------+---------+----------+----+-----+---+-----------------+--------------------+
only showing top 4 rows



In [46]:
loc_remove_space.shape()

(13200, 8)

In [47]:
df8=df7.withColumn("location",f.regexp_replace("location",'^\s+$','')) 

In [48]:
df8.shape()

(13200, 7)

In [49]:
df8.show(3)

+--------------------+---------+----------+----+-----+---+-----------------+
|            location|     size|total_sqft|bath|price|bhk|   price_per_sqft|
+--------------------+---------+----------+----+-----+---+-----------------+
|Electronic City P...|    2 BHK|    1056.0|   2|39.07|  2|3699.810606060606|
|    Chikka Tirupathi|4 Bedroom|    2600.0|   5|120.0|  4|4615.384615384615|
|         Uttarahalli|    3 BHK|    1440.0|   2| 62.0|  3|4305.555555555556|
+--------------------+---------+----------+----+-----+---+-----------------+
only showing top 3 rows



In [50]:
df8.orderBy("location").groupBy("location").count().show(3)

+------------+-----+
|    location|count|
+------------+-----+
|      Anekal|    1|
|   Banaswadi|    1|
| Basavangudi|    1|
+------------+-----+
only showing top 3 rows



In [51]:
from pyspark.sql.functions import desc

location_stats = df8.groupBy("location").count().sort(desc("count"))

location_stats.show(10)

+--------------------+-----+
|            location|count|
+--------------------+-----+
|          Whitefield|  532|
|      Sarjapur  Road|  392|
|     Electronic City|  302|
|      Kanakpura Road|  264|
|         Thanisandra|  232|
|           Yelahanka|  210|
|         Uttarahalli|  186|
|              Hebbal|  176|
|        Marathahalli|  175|
|Raja Rajeshwari N...|  171|
+--------------------+-----+
only showing top 10 rows



In [52]:
location_stats.printSchema()

root
 |-- location: string (nullable = true)
 |-- count: long (nullable = false)



In [53]:
location_stats.where("count <= 10").show(3)

+--------------------+-----+
|            location|count|
+--------------------+-----+
|    Dodsworth Layout|   10|
|            Basapura|   10|
|Nagappa Reddy Layout|   10|
+--------------------+-----+
only showing top 3 rows



In [54]:
location_stats_less_than_10 = location_stats.where("count <= 10")

location_stats_less_than_10.show(3)

+--------------------+-----+
|            location|count|
+--------------------+-----+
|    Dodsworth Layout|   10|
|            Basapura|   10|
|Nagappa Reddy Layout|   10|
+--------------------+-----+
only showing top 3 rows



In [55]:
df8.join(location_stats_less_than_10, df8.location == location_stats_less_than_10.location).count()

2887

In [56]:
#lambda x: 'other' if x in location_stats_less_than_10 else x)

def loc_stats_lt_10(x):
    if x in location_stats_less_than_10.location:
            return 'Other'
    else:
        return x
        